In [1]:
!pip install llama_index

In [2]:
!pip install PyPDF2

In [3]:
!pip install llama-index-embeddings-huggingface

In [4]:
!pip install llama-index-embeddings-huggingface-api

In [5]:
!pip install llama-index-llms-huggingface

In [6]:
!pip install llama-index-extractors-entity

In [7]:
!pip install nest_asyncio

In [8]:
!pip install -U span-marker

In [9]:
!pip install transformers==4.39

In [10]:
!pip install bitsandbytes

In [11]:
import nest_asyncio
import os

In [12]:
nest_asyncio.apply()

In [13]:
from google.colab import userdata, drive


In [14]:
#Transformer imports
from transformers import BitsAndBytesConfig
import transformers

In [15]:
#Pytorch import
import torch

In [16]:
#LlamaIndex imports
from llama_index.core import SimpleDirectoryReader, Settings, Document

from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter

from llama_index.core.ingestion import IngestionPipeline

from llama_index.extractors.entity import EntityExtractor #, TitleExtractor

from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index.core import SummaryIndex, VectorStoreIndex, ServiceContext

from llama_index.core.tools import QueryEngineTool

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
#from llama_index.core.llms import LLMPredictor


#from llama_index.llms import HuggingFaceLLM
#from llama_index.embeddings import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [17]:
from huggingface_hub import login

In [18]:
openApiToken = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openApiToken

In [19]:
huggingFaceToken = userdata.get('HF_TOKEN_DATAENLIGHT')
login(huggingFaceToken)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [20]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
defaultTextSplitter = SentenceSplitter(chunk_size=512, chunk_overlap = 256)
Settings.text_splitter = defaultTextSplitter

In [22]:
#Importing documents
pdfFilePath = "/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf"
pdfFileDirectory = "/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/"

pdfDocLs = SimpleDirectoryReader(pdfFileDirectory).load_data()

In [23]:
pdfDocLs[0]

Document(id_='3b062846-297f-4576-a7f9-bfe185290edd', embedding=None, metadata={'page_label': '1', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_type': 'application/pdf', 'file_size': 632296, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Online\nMaster\nof\nScience\nin\nComputer\nScience\n(OMSCS)\nSpring\n2024\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [24]:
docTransformationPipeline = IngestionPipeline(transformations=[])
documentNodes = docTransformationPipeline.run(documents = pdfDocLs)

In [25]:
documentNodes[8]

Document(id_='a2a5a8f0-87d4-48ca-b869-c3e42f94e369', embedding=None, metadata={'page_label': '9', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_type': 'application/pdf', 'file_size': 632296, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='SECTION\nI.\nSYSTEMS\nYOU\nWILL\nBE\nUSING\nAND\nWHY\nSystem\nWebsite\nReason\nOSCAR\nhttps://oscar.gatech.edu/\nUse\nthis\nto\nregister\nfor\nclasses,\npay\ntuition, \ncheck\nfor\nholds,\nview\nfinal\ngrades,\netc.\nCanvas\nhttps://canvas.gatech.edu/\nCanvas\nis\nwhere\n

In [26]:
tokenTextSplitter = TokenTextSplitter(chunk_size=512, chunk_overlap=256)
docTransformationPipeline = IngestionPipeline(transformations=[tokenTextSplitter])
testDocumentNodes = docTransformationPipeline.run(documents = pdfDocLs)

In [27]:
testDocumentNodes[8]

TextNode(id_='72448e59-7df1-4d23-af63-a80150acb701', embedding=None, metadata={'page_label': '3', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_type': 'application/pdf', 'file_size': 632296, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ccea9de0-47cc-476b-a6b7-55e638c20ffc', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '3', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab 

In [28]:
#Augmenting metadata with the entities identified in the Node's text (text chunk)
appliedExtractorsLs = []

appliedExtractorsLs.append(tokenTextSplitter)

# Provide the 'labels' argument to EntityExtractor
entityExtractor = EntityExtractor(
    #model_name = "tomaarsen/span-marker-bert-base-fewnerd-fine-super",
    model_name = "tomaarsen/span-marker-mbert-base-multinerd",
    prediction_threshold=0.6,
    label_entities = True,  # include the entity label in the metadata (can be erroneous)
    device="cuda",  # set to "cuda" if you have a GPU,
    labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"] # Labels for the NER model
)

appliedExtractorsLs.append(entityExtractor)

#titleExtractor = TitleExtractor(nodes = 5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
docTransformationPipeline = IngestionPipeline(transformations = appliedExtractorsLs)
extMetadataDocumentNodes = await docTransformationPipeline.arun(documents = pdfDocLs)

Extracting entities:   0%|          | 0/222 [00:00<?, ?it/s]

In [30]:
for node in extMetadataDocumentNodes:
    print(node.metadata)

{'page_label': '1', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_type': 'application/pdf', 'file_size': 632296, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-12'}
{'page_label': '2', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_type': 'application/pdf', 'file_size': 632296, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-12', 'organizations': ['College of Computing', 'Georgia Tech']}
{'page_label': '2', 'file_name': 'Orientation Documents-Spring 2024 Orientation Document.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/GATECH LLM Seminar/RAG Project/data/Orientation Documents-Sprin

LLM Setup - Llama3-7B-INSTRUCT

In [31]:
modelName = "meta-llama/Meta-Llama-3-8B-Instruct"
#modelName = "meta-llama/Meta-Llama-3-8B"

In [32]:
#Instatntiating LLAMA3 LLM
from transformers import AutoTokenizer

llama3Tokenizer = AutoTokenizer.from_pretrained(modelName)

stopping_ids = [
    llama3Tokenizer.eos_token_id,
    llama3Tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
llmParamTemperature = 0.2
llmParamTopP = 0.9

In [34]:
from transformers import BitsAndBytesConfig

In [35]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

#LQUANTIZATION TO BE ELABORATED LATER
# Optional quantization setup to 4bit
# import torch
#

#Quantization setup applied to achieve reasonable latency

quantization_config = BitsAndBytesConfig(
     load_in_4bit=True,
     bnb_4bit_compute_dtype=torch.float16,
     bnb_4bit_quant_type="nf4",
     bnb_4bit_use_double_quant=True,
)

llmModel = HuggingFaceLLM(
    model_name = modelName,
    #model_type = "llama",
    model_kwargs={
        #"token": huggingFaceToken,
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": llmParamTemperature,
        #"max_new_tokens": 1024,
        #"top_p": llmParamTopP,
    },
    tokenizer_name = modelName,
    #tokenizer_kwargs={"token": huggingFaceToken},
    stopping_ids=stopping_ids,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

text_embedding_model = "thenlper/gte-base"
embed_model = HuggingFaceEmbedding(model_name = text_embedding_model)

ragServiceContext = ServiceContext.from_defaults(
                                      llm = llmModel,
                                      embed_model = embed_model,
                                    )

<ipython-input-36-8eb5450c6a0e>:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  ragServiceContext = ServiceContext.from_defaults(


LLM Prompting

In [37]:
sampleResponse = llmModel.complete(["Describe biotechnology"])

print(sampleResponse)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Biotechnology is the use of biological systems, organisms, or biological processes to create or modify products, processes, or living organisms. It involves the use of biological tools, such as enzymes, microorganisms, and genes, to develop new products, treatments, and technologies. Biotechnology has a wide range of applications, including:

1. Medicine: Biotechnology is used to develop new medicines, vaccines, and treatments for diseases.
2. Agriculture: Biotechnology is used to develop genetically modified crops, improve crop yields, and reduce the use of pesticides and herbicides.
3. Environmental: Biotechnology is used to clean up pollution, restore ecosystems, and develop sustainable technologies.
4. Industrial: Biotechnology is used to develop new products, such as biofuels, bioplastics, and biochemicals.
5. Food: Biotechnology is used to develop new food products, such as genetically modified organisms (GMOs), and improve food safety and quality.

Biotechnology involves the us

#Constructing Summary and Detailed Vector Query Engine Tools for Orientation Document

In [39]:
orientationSummaryIndex = SummaryIndex(extMetadataDocumentNodes, service_context = ragServiceContext)
orientationVectorIndex = VectorStoreIndex(extMetadataDocumentNodes, service_context = ragServiceContext)

In [40]:
orientationSummaryQueryEngine = orientationSummaryIndex.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

orientationVectorQueryEngine = orientationVectorIndex.as_query_engine(use_async=True)

In [41]:
from llama_index.core.tools import QueryEngineTool

In [42]:
orientationSummaryTool = QueryEngineTool.from_defaults(
    query_engine = orientationSummaryQueryEngine,
    description=(
        "Tool useful for summarization of OMCS orientation document or general questions wide in scope concerning OMCS orientation"
    ),
)

orientationVectorTool = QueryEngineTool.from_defaults(
    query_engine = orientationVectorQueryEngine,
    description=(
        "Tool useful for retrieving specific context from OMCS orientation document"
    ),
)

Defining Router Query Engine

In [43]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

In [44]:
#Router Engine with OpenAI
routerQueryEngine = RouterQueryEngine(
                      selector=LLMSingleSelector.from_defaults(),
                      query_engine_tools=[
                              orientationSummaryTool,
                              orientationVectorTool,
                            ],
                      service_context = ragServiceContext,
                      verbose=True
                    )

In [45]:
response = routerQueryEngine.query("What is the summary of the OMCS orientation document?")
print(str(response))

Selecting query engine 0: The tool useful for summarization of OMCS orientation document or general questions wide in scope concerning OMCS orientation would be the most relevant choice for summarizing the OMCS orientation document..


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

 The OMCS orientation document is a comprehensive guide that provides students with information on various aspects of the Online Master of Science in Computer Science (OMSCS) program at Georgia Tech, including academic policies, registration, and student records. It covers topics such as course offerings, grade/GPA requirements, and policies related to course repetition, grade substitution, and academic standing. The document also explains the process of withdrawing from a course or semester, the impact of withdrawing on academic standing, and the importance of resolving incomplete grades. Additionally, it provides information on the waitlist process, course registration, and other relevant details for students. The document aims to inform and guide students through the program, ensuring a smooth and successful experience. Overall, the OMCS orientation document is a comprehensive guide that provides students with the information they need to succeed in the OMSCS program.


In [ ]:
from span_marker import SpanMarkerModel

model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-bert-base-fewnerd-fine-super")
model.cuda()

In [ ]:
for node in documentNodes[1:]:
  print(len(node.text))
  nodeEntities = model.predict("Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris.", )
  print(nodeEntities)

In [ ]:
for node in documentNodes[1:]:
  print(len(node.text))
  # Tokenize the input text
  inputs = model.tokenizer(
      "Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris.",
      #return_tensors="pt"
  )
  # Move inputs to the same device as the model
  inputs = {key: value.to(model.device) for key, value in inputs.items()}
  # Predict entities
  nodeEntities = model.predict(**inputs)
  print(nodeEntities)